In [ ]:
# http://stackoverflow.com/questions/44544766/ddg#44547144
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
from matplotlib import pyplot as plt
import keras

In [ ]:
def pd_read_csv(fn:str):
    x = pd.read_csv(fn, na_values=['NO_LABEL', '(blank)'])
    assert not x['Unnamed: 0'].duplicated().any()
    x = x.set_index("Unnamed: 0")
    return x

## Read data

In [ ]:
train = pd_read_csv('data_in/TrainingData.csv')
test = pd_read_csv('data_in/TestData.csv')

In [ ]:
# train.columns
# train['Position_Type'].head()

In [ ]:
features = list(set(train.columns).intersection(set(test.columns)) - set(['FTE','Total']))
features.sort()
features

In [ ]:
target = set(train.columns) - set(test.columns)
target = list(target)
target.sort()
target

In [ ]:
for col in target:
    test[col] = np.nan

In [ ]:
train.shape, test.shape

In [ ]:
train['is_holdout'] = False
test ['is_holdout'] = True
df = pd.concat([train,test], axis=0)
df.shape

In [ ]:
# convert FTE to string boolean and append to list of features
# df['FTE'] = ~pd.isnull(df['FTE']).astype('str')
df['FTE'] = df['FTE'].apply(lambda x: 'nan' if pd.isnull(x) else ( str(round(x,1)) if x <=1 else '>1' ) )
features = features + ['FTE']
features.sort()

In [ ]:
# categorize Total field
def total_to_category(x):
    if pd.isnull(x): return 'nan'
    # if x < 1000: return str(round(x,1)) if x <=1 else '>1' ) )
    ranges = [10,100,1000,10000, 1e5]
    for i in ranges:
        if abs(x) < i: return str(int(x//(i//10)*(i//10)))
        
    return "> %s"%str(int(max(ranges)))

{x: total_to_category(x) for x in [1.5, 3.43, 15, 153, 2153, 9123, 42153, 142153]}

In [ ]:
df['Total_sign'] = df['Total'].apply(lambda x: 'nan' if pd.isnull(x) else ('0' if x==0 else ('+' if x>0 else '-')))
df['Total_bin'] = df['Total'].apply(lambda x: total_to_category(x))
features = features + ['Total_sign', 'Total_bin']
features.sort()

In [ ]:
df['Total_sign'].value_counts().head(n=20)

In [ ]:
df['Total_bin'].value_counts().shape

In [ ]:
df['Total_bin'].value_counts().head(n=20)

In [ ]:
features

In [ ]:
meta = list(set(df.columns) - set(features) - set(target))
meta

In [ ]:
df.shape, df[features].shape, df[target].shape, df[meta].shape

## Analyze how close the train and test features are

In [ ]:
results = []
for ff in features:
    vc_train = df[ff][~df['is_holdout']].value_counts()
    vc_test  = df[ff][ df['is_holdout']].value_counts()
    # vc_train.shape, vc_test.shape
    vc_both  = vc_train.reset_index().merge(
        vc_test.reset_index(), 
        left_on = 'index', 
        right_on='index', 
        how='outer', 
        suffixes=['_train', '_test']
    )
    vc_both = vc_both.set_index('index')
    # vc_both.head()
    # vc_both[pd.isnull(vc_both['Facility_or_Department_test'])].head()
    out = {
        'feature': ff,
        'train all': df[~df['is_holdout']].shape[0],
        # 'train': vc_both['%s_train'%ff].sum(),
        'train non-null': (~pd.isnull(df[ff][~df['is_holdout']])).sum(),
        'train_minus_test': vc_both['%s_train'%ff][pd.isnull(vc_both['%s_test'%ff ])].sum(), 
        'test_minus_train': vc_both['%s_test'%ff ][pd.isnull(vc_both['%s_train'%ff])].sum(),
    }
    out['tmt_pct'] = out['test_minus_train'] * 100 // out['train non-null']
    results.append(out)


results = pd.DataFrame(results)
results = results.set_index('feature').sort_index()
results = results.astype('uint32')

In [ ]:
# results.shape
# results.head()
results[['train all', 'train non-null', 'train_minus_test', 'test_minus_train', 'tmt_pct']]

In [ ]:
# sod = train['Sub_Object_Description'].value_counts()
sod = test['Sub_Object_Description'].value_counts()

In [ ]:
sod.head(n=20)

In [ ]:
from matplotlib import pyplot as plt
# plt.bar(x=range(sod.shape[0]), height=sod.values)
plt.bar(x=range(sod.shape[0]-5), height=sod.iloc[5:].values)
plt.show()

In [ ]:
# sod[sod<10].shape[0], sod.shape[0]
sod[sod<10]

In [ ]:
subtest = test['Sub_Object_Description'].apply(lambda x: (~pd.isnull(x)) & ('community' in str(x).lower())) # .sum()

In [ ]:
test['Sub_Object_Description'][subtest].head()

## Read target labels

In [ ]:
import yaml
labels = yaml.load(open("labels.yml",'r'))

In [ ]:
# Function': ['Aides Compensation
prediction_names = []
for k,v1 in labels.items():
    for v2 in v1:
        pn = "%s__%s"%(k,v2)
        prediction_names.append(pn)
        
        
assert 'Function__Aides Compensation' in prediction_names
prediction_names.sort()
prediction_names[:5]

## one-hot encode each target by its classes

In [ ]:
for p in prediction_names: df[p] = False

In [ ]:
for k,v1 in labels.items():
    for v2 in v1:
        pn = "%s__%s"%(k,v2)
        # print(pn)
        df[pn] = df[k] == v2

In [ ]:
# since NO_LABEL is replaced with NaN, need this
for dependent in labels.keys():
    target_sub = [x for x in df.columns if x.startswith("%s__"%dependent)]
    df.loc[~df[target_sub].any(axis=1), '%s__NO_LABEL'%dependent]=True

In [ ]:
df[['Function', 'Function__Teacher Compensation', 'Function__Substitute Compensation', 'Function__NO_LABEL']].head()

In [ ]:
df.shape, df[pd.isnull(df[prediction_names]).all(axis=1)].shape, df.loc[~df[prediction_names].any(axis=1)].shape

In [ ]:
assert ~pd.isnull(df[prediction_names]).any().any()

In [ ]:
df[prediction_names] = df[prediction_names].astype('uint8')

## Factorize features

In [ ]:
print(time.ctime())
df_feat = df[features].apply(lambda x: pd.factorize(x)[0], axis=0)
df_feat = df_feat + 1 # +1 for the -1 from pd.factorize on nan (keras Embedding supports [0,N) )    
print(time.ctime())

In [ ]:
df_feat.max().max(), df_feat.min().min()

In [ ]:
vocab_size = df_feat.max(axis=0) + 1 # +1 to count the 0 index
vocab_size = vocab_size.sort_index()
vocab_size

In [ ]:
assert df[prediction_names].max().max()==1

In [ ]:
df['Total_sign'].value_counts()#.tail()#head(n=100).tail()

## split the non-holdout into train/test


In [ ]:
# calculate label_keys array whose order is replicable
label_keys = labels.keys()
label_keys = list(label_keys)
label_keys.sort()


In [ ]:
df_x = df_feat[~df['is_holdout']]
df_y = df[prediction_names][~df['is_holdout']] # .fillna(0)

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold

rskf = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=36851234)

## build a dummy equi-probable target

In [ ]:
def get_y_equi(n):
    y_equi = {}
    for k in label_keys:
        y_equi[k] = np.ones(shape=(n, len(labels[k]))) / len(labels[k])

    y_equi = [y_equi[k] for k in label_keys]
    y_equi = np.concatenate(y_equi, axis=1)
    y_equi = pd.DataFrame(y_equi, columns=y_train.columns, index=y_train.index)
    # y_equi.shape
    
    return y_equi

## keras embedding + Dense/LSTM

In [ ]:
from keras.layers import Embedding, Dense, Flatten, LSTM, Input, Concatenate, Add, Lambda, Dropout
from keras.models import Sequential, Model
from keras import backend as K


def build_fn():
    # vocab_size = stats.shape[0]

    # inputs = [Input(shape=(prob3.shape[1],)) for f in vocab_size.index]
    inputs = {f: Input(shape=(1,), name=f) for f in vocab_size.index}

    # embeddings = [Embedding(vocab_size[f], embedding_dim, input_length=prob3.shape[1]) for f in vocab_size.index]

    if True:
        embedding_dim = 10 # 3 # 12 # 2 # 64 # FIXME
        embeddings = {f: Embedding(vocab_size[f], embedding_dim, input_length=1)(inputs[f]) for f in vocab_size.index}
    else:
        embeddings = {f: Embedding(vocab_size[f], max(3, vocab_size[f]//15//10), input_length=1)(inputs[f]) for f in vocab_size.index}

    # the model will take as input an integer matrix of size (batch, input_length).
    # the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
    # now model.output_shape == (None, input_length, embedding_dim), where None is the batch dimension.

    # dummy variable
    x1= embeddings

    #  flatten each feature since no sequences anyway
    x1 = {f: Flatten(name="%s_flat"%f)(x1[f]) for f in vocab_size.index}

    # dense layer for each feature
    # x1 = {f: Dense(10, activation = 'relu', name="%s_d01"%f)(x1[f]) for f in vocab_size.index}
    # x1 = {f: Dense( 3, activation = 'relu', name="%s_d02"%f)(x1[f]) for f in vocab_size.index}

    # a dropout for each feature, this way, the network is more robust to dependencies on a single feature
    x1 = {f: Dropout(0.3, name="%s_dropout"%f)(x1[f]) for f in vocab_size.index}

    x1 = [x1[f] for f in vocab_size.index]
    x1 = Concatenate()(x1)
    # x1 = Flatten()(x1)
    x1 = Dropout(0.3)(x1)

    x1 = Dense(1000, activation='relu')(x1)
    x1 = Dense( 300, activation='relu')(x1)

    # x1 = Dense( 50, activation='relu')(x1)
    # o1 = {dependent: Dense(50, activation = 'relu', name="%s_d1"%dependent)(x1) for dependent in label_keys}
    # o1 = {dependent: Dense(50, activation = 'relu', name="%s_d2"%dependent)(o1[dependent]) for dependent in label_keys}

    # outputs = [Dense(len(labels[dependent]), activation = 'softmax', name="%s_out"%dependent)(o1[dependent]) for dependent in label_keys]
    outputs = [Dense(len(labels[dependent]), activation = 'softmax', name="%s_out"%dependent)(x1) for dependent in label_keys]

    inputs = [inputs[f] for f in vocab_size.index]
    model = Model(inputs=inputs, outputs=outputs)
    
    # model.compile('rmsprop', loss=multi_multiclass_logloss, metrics=['acc'])
    # model.compile('rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    model.compile('adam', loss='categorical_crossentropy', metrics=['acc'])
    
    return model

In [ ]:
model_test = build_fn()
model_test.summary()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import time

In [ ]:
models_k = []

# instead of df_y, use y_zeros below
# otherwise will get error 
# "ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead."
y_zeros = np.zeros(shape=(df_x.shape[0], 1))

for k, indeces in enumerate(rskf.split(df_x.values, y_zeros)):
    print('%s .. fold %s'%(time.ctime(), k))
    train_index, test_index = indeces
    
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = df_x.iloc[train_index], df_x.iloc[test_index]
    y_train, y_test = df_y.iloc[train_index], df_y.iloc[test_index]
    
    y_equi = get_y_equi(y_train.shape[0])
    
    # convert 2-D matrix of features into array of 1-D features
    # This is needed because each feature has a different vocabulary for its embedding
    x_train = [x_train[f].values for f in vocab_size.index]
    x_test  = [x_test [f].values for f in vocab_size.index]
    
    # convert 2-D matrix of targets into K arrays of C-D matrices 
    # where C is the number of classes of each target
    y_train = [y_train[[x for x in prediction_names if x.startswith("%s__"%f)]].values for f in label_keys]
    y_test  = [y_test [[x for x in prediction_names if x.startswith("%s__"%f)]].values for f in label_keys]
    y_equi  = [y_equi [[x for x in prediction_names if x.startswith("%s__"%f)]].values for f in label_keys]
    
    # len(y_train), y_train[0].shape, y_train[1].shape, len(y_test), y_test[0].shape, len(y_equi), y_equi[0].shape
    
    # y_equi[0][:2], y_equi[1][:2]
    
    model = build_fn()
    
    print('first train to equi-probable')
    model.fit(
        x_train,
        y_equi,
        batch_size=32*32, # 32, # FIXME
        epochs=5,
        verbose=0,#2,
        validation_split = 0.2,
        # validation_split = 0,
        shuffle=True
    )
    
    # y_pred = model.predict(x_train, batch_size=32*32)
    y_pred = model.predict(x_test,  batch_size=32*32)
    assert abs(y_pred[0][0,0] - 0.027) < .001
    assert abs(y_pred[1][0,0] - 0.090) < .001
    
    # evaluate on the real data
    score = model.evaluate(x_train, y_train, batch_size = 32*32)
    assert abs(score[0] - 18.69) < 0.01
    
    print('then train to actual probabilities')
    history = model.fit(
        # pd.get_dummies(train3['x'].values),
        # # train2[list(set(train2.columns) - set(['joined']))],
        # train3['y'].values,
        x_train,
        y_train,
        batch_size=32*32, # 32, # FIXME
        epochs=30,
        #initial_epoch=30,
        verbose=0, #2,
        validation_split = 0.2,
        # validation_split = 0,
        shuffle=True
    )
    
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    
    print('model.evaluate', model.evaluate(x_test, y_test, batch_size = 32*32))
    
    models_k.append(model)
    
    print('')
    print('')
    print('')

## Predict complete dataset for visualization

In [ ]:
len(models_k)

In [ ]:
def predict_k_models(x_in):
    y_pred = []
    n = len(models_k)
    for k,model in enumerate(models_k):
        print('fold %s / %s'%(k,n))
        y_pred.append(model.predict(x_in, verbose=2))
        
    # y_pred = pd.Panel(y_pred).mean(axis=2)
    
    # TODO replace pd.Panel with xarray
    # http://xarray.pydata.org/en/stable/
    y_out = []
    for fi in range(len(label_keys)):
        # print(fi)
        y_out.append(pd.Panel([yyy[fi] for yyy in y_pred]).mean(axis=0))
        
    for i,k in enumerate(label_keys):
        y_out[i].columns = labels[k]
        
    return y_out

In [ ]:
x_test = df_x # .head()
y_test = df_y # .head()

x_test  = [x_test [f].values for f in vocab_size.index]
y_test  = [y_test [[x for x in prediction_names if x.startswith("%s__"%f)]].values for f in label_keys]

y_pred = predict_k_models(x_test)

# y_pred = pd.Panel(y_pred).mean(axis=2)
# y_pred.shape

In [ ]:
y_test[0][0,:].sum(), y_pred[0].iloc[0,:].sum() # , y_pred[0]

## Spatial comparison

In [ ]:
# sub_labels = labels
sub_labels = {k:labels[k] for k in label_keys if k in ['Function']}

n_show = 1000

for i,v0 in enumerate(sub_labels.items()):
    k,v1 = v0
    # y_pred2 = pd.DataFrame(y_pred[i], columns=v1)
    y_pred2 = y_pred[i]
    y_test2 = pd.DataFrame(y_test[i], columns=v1)
    for v2 in v1:
        plt.figure(figsize=(20,3))
        plt.plot(y_pred2.loc[:n_show,v2], label='pred')
        #plt.plot(sum_pred, label='sum_pred', alpha=0.2)
        plt.plot(y_test2.loc[:n_show,v2], '.', label='actual')
        plt.legend(loc='best')
        plt.title("%s: %s"%(k,v2))

        axes = plt.gca()
        axes.set_ylim([-.1,1.1])
        plt.show()

## temporal comparison

In [ ]:
y_pred2 = {label_keys[i]: y_pred[i] for i in range(len(label_keys))}
y_test2 = {label_keys[i]: y_test[i] for i in range(len(label_keys))}

k2 = 'Function'
y_pred3 = y_pred2[k2].values
y_test3 = y_test2[k2]

for i in range(15):
    plt.figure(figsize=(10,3))
    
    plt.subplot(121)
    plt.bar(x=range(y_pred3.shape[1]), height=y_test3[i])
    plt.title('%s. actual, argmax=%s'%(i,np.argmax(y_test3[i])))
    axes = plt.gca()
    axes.set_ylim([-.1,1.1])
    
    plt.subplot(122)
    plt.bar(x=range(y_pred3.shape[1]), height=y_pred3[i])
    plt.title('%s. prediction, argmax=%s'%(i,np.argmax(y_pred3[i])))
    axes = plt.gca()
    axes.set_ylim([-.1,1.1])
    
    # plt.title(y_test.index[i])
    
    plt.show()

In [ ]:
i=6
y_test3[i,:]

In [ ]:
from keras.losses import categorical_crossentropy
from keras import backend as K

result_all = []
for yi in range(len(y_test)):
    result_i = categorical_crossentropy(K.variable(y_test[yi]), K.variable(y_pred[yi].values))
    result_all.append(K.eval(result_i))

In [ ]:
# len(result_all), result_all[0].shape
# result_all[0].mean()
# result_all[0][0:5], result_all[1][0:5], len(y_test)
pd.Series([r.mean() for r in result_all]).sum()

## Mock submission

In [ ]:
x_ho = df_feat[features][~df['is_holdout']].head()
x_ho  = [x_ho [f].values for f in vocab_size.index]
y_ho = predict_k_models(x_ho)
df_submit = pd.DataFrame(np.concatenate(y_ho, axis=1), columns=prediction_names)
df_submit.shape

In [ ]:
df_submit.head().round(1)

In [ ]:
df[target].head()

## Prepare submission

In [ ]:
df.shape, df_feat.shape

In [ ]:
x_ho = df_feat[features][ df['is_holdout']]
x_ho  = [x_ho [f].values for f in vocab_size.index]
y_ho = predict_k_models(x_ho)

In [ ]:
len(y_ho), y_ho[0].shape, y_ho[1].shape

In [ ]:
df_submit = pd.DataFrame(np.concatenate(y_ho, axis=1), columns=prediction_names, index=df_feat[ df['is_holdout']].index)
df_submit.shape

In [ ]:
df_submit.head()

In [ ]:
# plt.plot(df_submit['Use__NO_LABEL'].sort_values().values)
plt.plot(df_submit['Operating_Status__NO_LABEL'].sort_values().values)
plt.show()

In [ ]:
test.head()

In [ ]:
assert (df_submit['Operating_Status__NO_LABEL']<0.0001).all()
del df_submit['Operating_Status__NO_LABEL']

In [ ]:
fn = 'data_out/submission_D2_%s.csv'%(time.strftime("%Y%m%d_%H%M%S"))
df_submit.to_csv(fn)